<h1>Implementing Decision Tree ID3 Algorithm</h1>

The ID3 algorithm was developed by Quinlan in 1986, in 1993 Quinlan published its successor C4.5. ID3 will perform following tasks recursively:
<ul>
<li>Create a root node for the tree
<li>If all examples are positive, return leaf node ‘positive’
<li>Else if all examples are negative, return leaf node ‘negative’
<li>Calculate the entropy of current state E(S)
<li>For each attribute, calculate the entropy with respect to the attribute ‘A’ denoted by E(S, A)
<li>Select the attribute which has the maximum value of IG(S, A) and split the current (parent) node on the selected attribute
<li>Remove the attribute that offers highest IG from the set of attributes
<li>Repeat until we run out of all attributes, or the decision tree has all leaf nodes.
</ul>

In [1]:
import numpy as np 
import pandas as pd 
import math
import time
import random
toPrint=False

<h3>Utility functions</h3>

In [2]:
def printTree(dic, nspace=0):
    nspace+=2
    if type(dic) is dict:
        for key in dic.keys():
            if type(key) is int:
                print(key, ":", end='', sep='')
            else:
                print("\n", ' '*nspace, key, ": ", end='', sep='')
            printTree(dic[key], nspace)
    else:
        print(dic, ",  ", end='', sep='')
    
dic1={'Outlook': {'sunny': {'Humidity': {'<0.72': 'p'}}, 'overcast': 'p', 'rain': {'Windy': {False: 'p', True: 'n'}}}}
printTree(dic1)
print()

def KNNresultTable(y_pred, y_test):
    XP=pd.DataFrame({'pred':y_pred, 'test':y_test, 'one':1})
    pt=XP.pivot_table(values='one', columns='pred', index='test',  aggfunc='count')
    pt=pt.replace(np.nan, 0)
    pt=pt.sort_index(axis=0,ascending=False).sort_index(axis=1,ascending=False)
    print()
    if (pt.shape[0]>1) and (pt.shape[1])>1:
        print("Accuracy:",(pt.iloc[0,0]+pt.iloc[1,1])/(pt.iloc[0,0]+pt.iloc[0,1]+pt.iloc[1,0]+pt.iloc[1,1]))
        print("Recall:",pt.iloc[0,0]/(pt.iloc[0,0]+pt.iloc[0,1]))
        print("Precision:",pt.iloc[0,0]/(pt.iloc[0,0]+pt.iloc[1,0]))
        print("F1 score:",pt.iloc[0,0]*2/(pt.iloc[0,0]*2+pt.iloc[0,1]+pt.iloc[1,0]))
    return pt

def sample(df, testRatio):
    selector=np.array([random.uniform(0,1)>=testRatio for i in range(df.shape[0])])
    X_train=df[selector]
    X_train.index=range(X_train.shape[0])
    X_test=df[selector==False]
    X_test.index=range(X_test.shape[0])
    return X_train, X_test 

df= pd.read_csv('playTennis.csv')
X_train, X_test=sample(df, 0.3)
print("tarin set shape:", X_train.shape)
print("test set shape:", X_test.shape)



  Outlook: 
    sunny: 
      Humidity: 
        <0.72: p,  
    overcast: p,  
    rain: 
      Windy: 
        False: p,  
        True: n,  
tarin set shape: (13, 5)
test set shape: (1, 5)


<h3>Basic functions for Decision Tree</h3>

In [3]:
def entropy(df):
    col=list(df[classVar])
    values=set(col)
    if len(values)<=1: return 0
    ntotal=df.shape[0]
    ent=0.0
    for v in values:
        ratio=float(col.count(v)/ntotal)
        ent+=ratio*math.log(ratio,2)
    return -ent

def testSplitNodes(df, attrib):
    values=list(set(df[attrib]))
    outDFs=[]
    for v in values:
        subdf=df[df[attrib]==v]
        outDFs.append(subdf)
    return values, outDFs

def trySplit(df,attrib):
    values, nodes=testSplitNodes(df, attrib)
    ntotal=df.shape[0]
    ent=0.0
    for i in range(len(values)):
        nodei=nodes[i]
        ent+=float(nodei.shape[0]/ntotal)*entropy(nodei)
    return ent


<h3>A non-recursive approach</h3>

This design put all the splitted data to the same array, it is computationally OK but is not easy for the prediction stage.

In [4]:
def splitNodes(df_entropy, attrib):
    df=df_entropy[0]
    NodeSignature=df_entropy[2]
    if toPrint: print("----- split by ",attrib, "----")
    values=list(set(df[attrib]))
    if len(values)==0: return
    if len(values)==1: 
        NodeSignature[attrib]=values[0]
        decisionTree.append((NodeSignature, df[classVar][0], df.shape[0]))
        return
    for v in values:
        subdf=df[df[attrib]==v].drop(attrib, axis=1)
        theEntropy=entropy(subdf)
        NodeSignature[attrib]=v
        if theEntropy!=0: 
            if toPrint: 
                print(v, theEntropy,NodeSignature)
                print(subdf)
            allnodes.append((subdf, theEntropy, NodeSignature.copy()))
        else:
            decisionTree.append((NodeSignature.copy(), list(subdf[classVar])[0], subdf.shape[0]))

df=pd.read_csv('playTennis.csv')
classVar='Decision'
totalEntropy=entropy(df)

allnodes=[]
allnodes.append([df, totalEntropy, dict()])

decisionTree=[]

toPrint=False

while len(allnodes)>0:
    DF=allnodes[0][0]
    TheEntropy=entropy(DF)
    InfoGains=pd.Series()
    for v in DF.columns:
        if v !=classVar:
            InfoGains[v]=TheEntropy-trySplit(DF,v)
    attrib=list(InfoGains.index[InfoGains==InfoGains.max()])[0]
    #print(InfoGains)
    print(attrib)
    splitNodes(allnodes[0],attrib)
    del allnodes[0]
    
decisionTree

Outlook
Windy
Humidity


[({'Outlook': 'overcast'}, 'p', 4),
 ({'Outlook': 'rain', 'Windy': False}, 'p', 3),
 ({'Outlook': 'rain', 'Windy': True}, 'n', 2),
 ({'Outlook': 'sunny', 'Humidity': 'high'}, 'n', 3),
 ({'Outlook': 'sunny', 'Humidity': 'normal'}, 'p', 2)]

<h3>Build the tree recursively</h3>

For the simplicity of prediction, we want wo have pure tree structure so we can traverse it recursively. The previous approach is not effective in the prediction stage

In [7]:
#what kind of data structure should the Decision Tree be? 
a={'a':{}}
a['a']['a1']=1
a['a']['a2']=2

b={'b':{}}
b['b']['b1']=10
b['b']['b2']=20

d={'d':{}}
d['d']['d1']=b
d['d']['d2']=2.2
d['d']['d3']=a
d

{'d': {'d1': {'b': {'b1': 10, 'b2': 20}},
  'd2': 2.2,
  'd3': {'a': {'a1': 1, 'a2': 2}}}}

In [8]:
printTree(d)


  d: 
    d1: 
      b: 
        b1: 10,  
        b2: 20,  
    d2: 2.2,  
    d3: 
      a: 
        a1: 1,  
        a2: 2,  

In [9]:
def getMostInfogainAttrib(df):
    if df.shape[1]==1: return df.columns[0]
    TheEntropy=entropy(df)
    InfoGains=pd.Series()
    for v in df.columns:
        if v !=classVar:
            InfoGains[v]=TheEntropy-trySplit(df,v)
    return list(InfoGains.index[InfoGains==InfoGains.max()])[0]

def splitNodes(df, treeNode):
    attrib=getMostInfogainAttrib(df)
    theEntropy=entropy(df)
    treeNode[attrib]=dict()
    if toPrint: print("----- split by ",attrib, "----")
    values=list(set(df[attrib]))
    if len(values)==0: return
    if len(values)==1: 
        if toPrint: print("all nodes are the same. Output")
        treeNode[attrib][values[0]]=list(df[classVar])[0]
        return
    for v in values:
        subdf=df[df[attrib]==v].drop(attrib, axis=1)
        theEntropy=entropy(subdf)
        if theEntropy!=0: 
            if toPrint: 
                print(v, theEntropy)
                print(subdf)
            treeNode[attrib][v]={}
            splitNodes(subdf, treeNode[attrib][v])
        else:
            if toPrint: print(v," --leaf node: ",v,":",list(subdf[classVar])[0], "(",subdf.shape[0],")")
            treeNode[attrib][v]=list(subdf[classVar])[0]
    del df


In [10]:
toPrint=False
df=pd.read_csv('playTennis.csv')
classVar='Decision'

DecisionTree=dict()
splitNodes(df, DecisionTree)
DecisionTree

{'Outlook': {'rain': {'Windy': {False: 'p', True: 'n'}},
  'overcast': 'p',
  'sunny': {'Humidity': {'high': 'n', 'normal': 'p'}}}}

<h3>Prediction</h3>

If it done right, the prediction could be very effictive. It can also be very very slow and difficult. 

In [11]:
# to understand how recursive matching works, set up the test data and run next cell repeatly
X_test=df.iloc[0,:]
print(X_test)
printTree(DecisionTree)
dic=DecisionTree

Outlook        sunny
Temperature      hot
Humidity        high
Windy          False
Decision           n
Name: 0, dtype: object

  Outlook: 
    rain: 
      Windy: 
        False: p,  
        True: n,  
    overcast: p,  
    sunny: 
      Humidity: 
        high: n,  
        normal: p,  

In [12]:
## Each run of following code will drill down via the first child of each branch
if type(dic)==dict: 
    field=next(iter(dic))
    print(field, type(dic[field]), len(dic[field]))
    dic=dic[field].get(X_test[field])
    
print(dic)

Outlook <class 'dict'> 3
{'Humidity': {'high': 'n', 'normal': 'p'}}


In [13]:
# .get() function is very useful, it also handles the exception iternally
dic=DecisionTree
print(dic['Outlook'].get('sunny'))
print(dic['Outlook'].get('sunnyX'))
print(dic['Outlook'].get('sunnyX', 'abcd'))

{'Humidity': {'high': 'n', 'normal': 'p'}}
None
abcd


In [14]:
## testing with the specific value to see how the get() works
## it is better to use the .get() function since it allow a non existing value without caursing error
match=dic['Outlook'].get('sunny') #overcast' #OVERCAST   #X_test['Outlook']) 
if match is not None:
    if type(match) is dict:
        print(match)
    else:
        print(match, 'is a leaf')

{'Humidity': {'high': 'n', 'normal': 'p'}}


In [15]:
#the above code is close enough for a recursive prediction function, it take a dictionary and predict the testing point's 
def predict(dic, x_test):
    field=next(iter(dic))
    match=dic[field].get(x_test[field]) 
    if match is None: return
    if type(match) is dict:
        if toPrint: print(match)  # match is a shorted dictionary so it can be treated just like the dic
        return predict(match, x_test)
    else:
        if toPrint: print(match, 'is a leaf of type ', type(match))
        return match

X_test=df.iloc[0,:]
print(DecisionTree)
dic=DecisionTree
predict(dic, X_test)

{'Outlook': {'rain': {'Windy': {False: 'p', True: 'n'}}, 'overcast': 'p', 'sunny': {'Humidity': {'high': 'n', 'normal': 'p'}}}}


'n'

In [16]:
# since we use all the rows in the data file to build the decision tree and we did not limit the number of levels, 
# it should not be a suprise to see all the points matched
for i in range(df.shape[0]):
    print(predict(dic, df.iloc[i,:]), df.iloc[i,4])

n n
n n
p p
p p
p p
n n
p p
n n
p p
p p
p p
p p
p p
n n


<h3>Training and testing</h3>

In [17]:
#random.seed(1234)
t0=time.time()
df=pd.read_csv('playTennis.csv')
toPrint=False
iscontinuous={col:False for col in df}
classVar='Decision'

X_train, X_test=sample(df, 0.2)

decisiontree=dict()
splitNodes(X_train,decisiontree)
print(time.time()-t0, 's')
print("{'Outlook': {'sunny': {'Humidity': {'high': 'n', 'normal': 'p'}}, 'overcast': 'p', 'rain': {'Windy': {False: 'p', True: 'n'}}}}")
print(decisiontree)

predicted=[predict(decisiontree, X_test.iloc[i,:]) for i in range(X_test.shape[0])]

KNNresultTable(predicted, X_test[classVar])

for i in range(X_test.shape[0]):
    if predicted[i]!=X_test[classVar][i]:
        print(predicted[i], list(X_test.iloc[i,:]))

0.06327104568481445 s
{'Outlook': {'sunny': {'Humidity': {'high': 'n', 'normal': 'p'}}, 'overcast': 'p', 'rain': {'Windy': {False: 'p', True: 'n'}}}}
{'Temperature': {'cool': {'Outlook': {'rain': 'n', 'sunny': 'p'}}, 'mild': 'p', 'hot': {'Outlook': {'overcast': 'p', 'sunny': 'n'}}}}

Accuracy: 0.25
Recall: 0.5
Precision: 0.3333333333333333
F1 score: 0.4
n ['rain', 'cool', 'normal', False, 'p']
None ['overcast', 'cool', 'normal', True, 'p']
p ['sunny', 'mild', 'high', False, 'n']
p ['rain', 'mild', 'high', True, 'n']


In [18]:
nbad=0
for i in range(X_train.shape[0]):
    predicted=predict(decisiontree, X_train.iloc[i,:])
    if predicted!=X_train[classVar][i]:
        print("mismatch:",predicted!=X_train[classVar][i])
        nbad+=1
print(nbad, " is mismatched")

0  is mismatched


<h3>Testing with different dataset</h3>

In [19]:
#https://archive.ics.uci.edu/ml/machine-learning-databases/balance-scale/balance-scale.data', 
df=pd.read_csv('balance-scale.csv', sep= ',') 

t0=time.time()
toPrint=False
classVar='Class'
DecisionTree=dict()

splitNodes(df, DecisionTree)
print(time.time()-t0)
DecisionTree

1.4859800338745117


{'RightWeight': {1: {'LeftWeight': {1: {'LeftDistance': {1: {'RightDistance': {1: 'B',
        2: 'R',
        3: 'R',
        4: 'R',
        5: 'R'}},
      2: {'RightDistance': {1: 'L', 2: 'B', 3: 'R', 4: 'R', 5: 'R'}},
      3: {'RightDistance': {1: 'L', 2: 'L', 3: 'B', 4: 'R', 5: 'R'}},
      4: {'RightDistance': {1: 'L', 2: 'L', 3: 'L', 4: 'B', 5: 'R'}},
      5: {'RightDistance': {1: 'L', 2: 'L', 3: 'L', 4: 'L', 5: 'B'}}}},
    2: {'LeftDistance': {1: {'RightDistance': {1: 'L',
        2: 'B',
        3: 'R',
        4: 'R',
        5: 'R'}},
      2: {'RightDistance': {1: 'L', 2: 'L', 3: 'L', 4: 'B', 5: 'R'}},
      3: 'L',
      4: 'L',
      5: 'L'}},
    3: {'LeftDistance': {1: {'RightDistance': {1: 'L',
        2: 'L',
        3: 'B',
        4: 'R',
        5: 'R'}},
      2: 'L',
      3: 'L',
      4: 'L',
      5: 'L'}},
    4: {'LeftDistance': {1: {'RightDistance': {1: 'L',
        2: 'L',
        3: 'L',
        4: 'B',
        5: 'R'}},
      2: 'L',
      3: 'L',
  

In [20]:
printTree(DecisionTree)


  RightWeight: 1:
      LeftWeight: 1:
          LeftDistance: 1:
              RightDistance: 1:B,  2:R,  3:R,  4:R,  5:R,  2:
              RightDistance: 1:L,  2:B,  3:R,  4:R,  5:R,  3:
              RightDistance: 1:L,  2:L,  3:B,  4:R,  5:R,  4:
              RightDistance: 1:L,  2:L,  3:L,  4:B,  5:R,  5:
              RightDistance: 1:L,  2:L,  3:L,  4:L,  5:B,  2:
          LeftDistance: 1:
              RightDistance: 1:L,  2:B,  3:R,  4:R,  5:R,  2:
              RightDistance: 1:L,  2:L,  3:L,  4:B,  5:R,  3:L,  4:L,  5:L,  3:
          LeftDistance: 1:
              RightDistance: 1:L,  2:L,  3:B,  4:R,  5:R,  2:L,  3:L,  4:L,  5:L,  4:
          LeftDistance: 1:
              RightDistance: 1:L,  2:L,  3:L,  4:B,  5:R,  2:L,  3:L,  4:L,  5:L,  5:
          LeftDistance: 1:
              RightDistance: 1:L,  2:L,  3:L,  4:L,  5:B,  2:L,  3:L,  4:L,  5:L,  2:
      LeftWeight: 1:
          RightDistance: 1:
              LeftDistance: 1:R,  2:B,  3:L,  4:L,  5:L,  2:
     

In [44]:
import random

#random.seed(1234)
t0=time.time()
df=pd.read_csv('balance-scale.csv', sep= ',') 

X_train, X_test=sample(df, 0.1)
print(X_train.shape[0], 'rows in training set')

toPrint=False
iscontinuous={col:False for col in df}
classVar='Class'
decisiontree=dict()
splitNodes(X_train,decisiontree)
print(time.time()-t0, 's')
#print(decisiontree)

predicted=[predict(decisiontree, X_test.iloc[i,:]) for i in range(X_test.shape[0])]

KNNresultTable(predicted, list(X_test[classVar]))
df    

575 rows in training set
1.5693919658660889 s

Accuracy: 0.45161290322580644
Recall: 0.6086956521739131
Precision: 0.6363636363636364
F1 score: 0.6222222222222222


,Class,LeftWeight,LeftDistance,RightWeight,RightDistance
0,B,1,1,1,1
1,R,1,1,1,2
2,R,1,1,1,3
3,R,1,1,1,4
4,R,1,1,1,5
5,R,1,1,2,1
6,R,1,1,2,2
7,R,1,1,2,3
8,R,1,1,2,4
9,R,1,1,2,5


In [22]:
decisiontree

{'RightDistance': {1: {'LeftWeight': {1: {'RightWeight': {1: {'LeftDistance': {1: 'B',
        2: 'L',
        3: 'L',
        4: 'L',
        5: 'L'}},
      2: {'LeftDistance': {1: 'R', 2: 'B', 3: 'L', 4: 'L', 5: 'L'}},
      3: {'LeftDistance': {1: 'R', 2: 'R', 3: 'B', 4: 'L', 5: 'L'}},
      4: {'LeftDistance': {1: 'R', 2: 'R', 3: 'R', 4: 'B'}},
      5: 'R'}},
    2: {'LeftDistance': {1: {'RightWeight': {1: 'L', 3: 'R', 4: 'R', 5: 'R'}},
      2: {'RightWeight': {1: 'L', 2: 'L', 3: 'L', 4: 'B', 5: 'R'}},
      3: 'L',
      4: 'L',
      5: 'L'}},
    3: {'LeftDistance': {1: {'RightWeight': {1: 'L', 2: 'L', 3: 'B', 4: 'R'}},
      2: 'L',
      3: 'L',
      4: 'L',
      5: 'L'}},
    4: {'RightWeight': {1: 'L',
      2: 'L',
      3: 'L',
      4: {'LeftDistance': {1: 'B', 2: 'L', 3: 'L', 5: 'L'}},
      5: {'LeftDistance': {1: 'R', 2: 'L', 3: 'L', 4: 'L', 5: 'L'}}}},
    5: {'LeftDistance': {1: {'RightWeight': {1: 'L', 3: 'L', 4: 'L', 5: 'B'}},
      2: 'L',
      3: 'L',
     

In [45]:
L=df['LeftWeight']*df['LeftDistance']
R=df['RightWeight']*df['RightDistance']
S=df['Class']

In [46]:
print((L==R).sum())
print((L<R).sum())
print((L>R).sum())
print(S.value_counts())

49
288
288
L    288
R    288
B     49
Name: Class, dtype: int64


<h2>Remaining issues</h2>

<ul>
<li>How about using Gini impurity score for the measurements
<li>How about the continues variable
<li>Overfitting issue
<li>number of levels
<li>Avoid physical divide
</ul>

<h3>Gini Impurity Score</h3>

<b>Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset.</b>

Let $S$ be a set of data of size $n$ and with distinct label $(c_1, c_2,..,c_k)$, its Gini impurity is defined by:

$$
Gini(S)=\sum_{i\ne j} p(s=c_i) p(s=c_j)~~~~~~ 1 \leq i,j \leq k
$$

Using shorthand notation $p_i=p(s=c_i)=n_i/n$, the above definition can be expressed as:

$$
Gini(S)=\sum_{i\ne j} p_i p_j=\sum_{i=1}^k\sum_{j=1,j \neq i}^k p_i p_j=\sum_{i=1}^k p_i\sum_{j=1, j\neq i}^k p_j=
   \sum_{i=1}^k p_i(1-p_i) = 1-\sum_{i=1}^kp_i^2
$$

$Gini(S)$ will be 0 when all the values in $S$ are the same, and it reaches its maximum value when all values are eqally distributed.

In decision tree applications, Gini impurity is used as cost function to evaluate splits in the dataset. A split in the dataset involves one input attribute and <b>one value</b> for that attribute. 

A Gini impurity gives an idea of how good a split is by how mixed the classes are in the two groups created by the split. A perfect separation results in a Gini score of 0, whereas the worst case split that results in 50/50 classes in each group result in a Gini score of 0.5 (for a 2 class problem).


The rules for spliting dataset is very similar bwteeen using entropy measurement or Gini impurity measurement. they are just different 'cost score'
<ol>
<li>Try to divide the dataset to multiple subset so that each subset is 'pure' by the attrib.
<li>compute the cost score for each subset, and summarize into one score for that attribute
<li>repeat the step 1 and 2 for all attributes in the dataset and select the 'best' attribute to make a divide.
<li>the attribute used in the division need not to be kept, because its values in each subset are all the same
<li>repeat above step for all subsets and all attributes
</ol>

<h4>Cost-score of a split</h4>

We are examing how the cost score changes when the dataset $S$ is divided into $m$ subsets: $S=(S_1, S_2,...,S_m)$.

When entropy is used as the cost-score, the information gain is the criteria used to determind the split:

$$
Gain(S,A)=Entropy(S)-\sum_{j=1}^m \frac{N(S_j)}{N(S)}Entropy(S_j)
$$

Since the entropy of the dataset $S$ is a constant, to maximize the information gain Gain(S,A) is the same as minimize the weighted entropy 

$$ max~ Gain(S,A) <====> min~ \sum_{j=1}^m \frac{N(S_j)}{N(S)}Entropy(S_j)$$

When Gini impurity is used for the 'cost-score', the parallel definition is used:

$$
   Gini(s_1, s_2,...,s_m)=\sum_{i=1}^m \frac{N(s_i)}{N(S)}Gini(s_i)
$$

This is nice because both of them are very compatible in form. 

<h4>Categorical attrib vs continuous attributes</h4> 

So far we used only the categorical variable as attributes, we need to deal with the continous variable too. The solution is to use a threshold value to split the attribute to two groups. For the classification (decision) trees, the "class" is alway a categorical variable, the entropy is <b>always</b> calculated on the class label, not on the actual attributes, both entropy and Gini score can be used for either categorical variable or continous variables.

Note, not all numeric value are continous, in fact, numeric coding of a categorical variable is very effictive way in the computing. We can not depend solely on variable typing to distinguesh them. Explicit definition may be the safest.

<h4>Binary or mutiple branch</h4>

The binary tree or tree with multiple branches are not much different. They are considered computationaly equivalent. The better approaches in the decision tree classification is to let data determind the structure of the tree. 

<h3>Spliting Continuous Variable into groups</h3>

We now turn to use the continous variable in decision tree classification

In [25]:
# dataset X is needed for this section
X=pd.DataFrame(columns=['X1','X2','Y'],
               data=np.array([
[2.771244718, 1.784783929, 0],
[1.728571309, 1.169761413, 0],
[3.678319846, 2.81281357 , 0],
[3.961043357, 2.61995032 , 0],
[2.999208922, 2.209014212, 0],
[7.497545867, 3.162953546, 1],
[9.00220326 , 3.339047188, 1],
[7.444542326, 0.476683375, 1],
[10.12493903, 3.234550982, 1],
[6.642287351, 3.319983761, 1]]))
X['Y'].astype(int)

#define the classVar as the class variable
classVar='Y'

In [26]:
def test_split(df, attrib, value=np.NAN):
    col=df[attrib]
    outDFs=[]
    if (np.isnan(value)):
        for v in list(set(col)):
            outDFs.append(df[col==v])
    else:
        outDFs.append(df[col<=value])
        outDFs.append(df[col> value])
    return outDFs

groups=test_split(X,'X1', 7)

In [27]:
groups

[         X1        X2    Y
 0  2.771245  1.784784  0.0
 1  1.728571  1.169761  0.0
 2  3.678320  2.812814  0.0
 3  3.961043  2.619950  0.0
 4  2.999209  2.209014  0.0
 9  6.642287  3.319984  1.0,           X1        X2    Y
 5   7.497546  3.162954  1.0
 6   9.002203  3.339047  1.0
 7   7.444542  0.476683  1.0
 8  10.124939  3.234551  1.0]

In [28]:
## using pd.Series.value_counts() is slower since the function is too complicate 
def entropy(cls):
    values=set(cls)
    if len(values)<=1: return 0
    size=len(cls)
    score=0.0
    freq=cls.value_counts()
    for class_val in freq.index:
        p = freq[class_val]/size
        score += p * math.log(p,2)
    return -score

def gini(cls):
    values=set(cls)
    if len(values)<=1: return 0
    size=len(cls)
    score=0.0
    freq=cls.value_counts()
    for class_val in freq.index:
        p = freq[class_val]/size
        score += p * p
    return 1.0-score

classVar='Y'
print(entropy(X[classVar]))
print(gini(X[classVar]))


1.0
0.5


<b> plug in the Entropy or Gini function</b>  

In [29]:
# slightly different approach 
def entropy(df):
    col=list(df[classVar])
    values=set(col)
    if len(values)<=1: return 0
    size=df.shape[0]
    ent=0.0
    for v in values:
        ratio=float(col.count(v)/size)
        ent+=ratio*math.log(ratio,2)
    return -ent

def gini(df):
    col=list(df[classVar])
    values=set(col)
    if len(values)<=1: return 0
    size=df.shape[0]
    score=0.0
    for v in values:
        ratio=float(col.count(v)/size)
        score+=ratio*ratio
    return 1-score

print(entropy(X))
print(gini(X))

1.0
0.5


In [30]:
def overall_score(dfs):
    score=0.0
    overallsize=0
    for df in dfs:
        overallsize+=df.shape[0]
        if toPrint: print(df.shape[0], measurement(df))
        score+=float(df.shape[0]*measurement(df))
    return score/float(overallsize)

measurement=gini
toPrint=False
overall_score(groups)

0.1666666666666666

In [31]:
def make_split(df):
    b_attrib, b_value, b_score, b_groups = "", np.NAN, 999, None
    for attrib in df.columns:
        if attrib==classVar: continue
        for value in df[attrib]:
            groups=test_split(df, attrib, value)
            score=overall_score(groups)
            if score<b_score:
                b_attrib, b_value, b_score, b_groups = attrib, value, score, groups
    return b_attrib, b_value, b_groups, b_groups

make_split(X)

('X1', 3.961043357, [         X1        X2    Y
  0  2.771245  1.784784  0.0
  1  1.728571  1.169761  0.0
  2  3.678320  2.812814  0.0
  3  3.961043  2.619950  0.0
  4  2.999209  2.209014  0.0,           X1        X2    Y
  5   7.497546  3.162954  1.0
  6   9.002203  3.339047  1.0
  7   7.444542  0.476683  1.0
  8  10.124939  3.234551  1.0
  9   6.642287  3.319984  1.0], [         X1        X2    Y
  0  2.771245  1.784784  0.0
  1  1.728571  1.169761  0.0
  2  3.678320  2.812814  0.0
  3  3.961043  2.619950  0.0
  4  2.999209  2.209014  0.0,           X1        X2    Y
  5   7.497546  3.162954  1.0
  6   9.002203  3.339047  1.0
  7   7.444542  0.476683  1.0
  8  10.124939  3.234551  1.0
  9   6.642287  3.319984  1.0])

<h3>Avoid physically split during the test stage</h3>

The above algorithm uses large amount of physically build the split before calculating the Entropy, this can be avoided because the calculation of entropy or information gain uses only the class variable. 


In [32]:
## redefine the entropy and gini index so that it deals only with the class variable
def Entropy(cls):
    cls=list(cls)
    values=set(cls)
    if len(values)<=1: return 0
    size=len(cls)
    score=0.0
    for v in values:
        p = cls.count(v)/size
        score += p * math.log(p,2)
    return -score

def Gini(cls):
    cls=list(cls)
    values=set(cls)
    if len(values)<=1: return 0
    size=len(cls)
    score=0.0
    for v in values:
        p = cls.count(v)/size
        score += p * p
    return 1.0-score

classVar='Y'
print(Entropy(X[classVar]))
print(Gini(X[classVar]))


1.0
0.5


In [33]:
def overall_score_no_split(df, attrib, value=np.NAN):
    col=df[attrib]
    cls=df[classVar]
    keys=list(set(col))
    overallsize=0
    score=0.0
    if np.isnan(value):
        for v in keys:
            selector=col==v
            sizei=sum(selector)
            overallsize+=sizei
            score+=measurement(cls[selector])*sizei
    else:
        selector=col<=value
        sizei=float(sum(selector))
        overallsize+=sizei
        score+=measurement(cls[selector])*sizei
        
        selector=selector==False
        sizei=float(sum(selector))
        overallsize+=sizei
        score+=measurement(cls[selector])*sizei
    return score/float(overallsize)

classVar='Y'
measurement=Gini
toPrint=False
overall_score_no_split(X, 'X1', 7)

0.1666666666666666

In [34]:
def find_best_split(df):
    b_attrib, b_score, b_value = "", 999, np.NAN
    for attrib in df.columns:
        if attrib==classVar: continue
        if iscontinuous[attrib]:
            for value in df[attrib]:
                score=overall_score_no_split(df, attrib, value)
                if toPrint: print(attrib, score, value)
                if score<b_score:
                    b_attrib, b_score, b_value= attrib, score, value
        else:
            score=overall_score_no_split(df, attrib)
            if score<b_score:
                b_attrib, b_score, b_value = attrib, score, np.NAN
        if toPrint: print(attrib, score, b_attrib, b_score, b_value)
    return b_attrib, b_score, b_value

iscontinuous={col:False for col in X}
iscontinuous['X1']=True
iscontinuous['X2']=True
classVar='Y'
toPrint=True
find_best_split(X)

X1 0.375 2.771244718
X1 0.4444444444444445 1.728571309
X1 0.1666666666666666 3.678319846
X1 0.0 3.961043357
X1 0.2857142857142857 2.999208922
X1 0.375 7.497545867
X1 0.4444444444444445 9.00220326
X1 0.2857142857142857 7.444542326
X1 0.5 10.12493903
X1 0.1666666666666666 6.642287351
X1 0.1666666666666666 X1 0.0 3.961043357
X2 0.4761904761904763 1.784783929
X2 0.5 1.169761413
X2 0.1666666666666666 2.81281357
X2 0.31999999999999984 2.61995032
X2 0.41666666666666663 2.209014212
X2 0.2857142857142857 3.162953546
X2 0.5 3.339047188
X2 0.4444444444444445 0.476683375
X2 0.375 3.234550982
X2 0.4444444444444445 3.319983761
X2 0.4444444444444445 X1 0.0 3.961043357


('X1', 0.0, 3.961043357)

In [35]:
df=pd.read_csv('playTennis_N.csv')
print(df)
iscontinuous={col:False for col in df}
iscontinuous['Humidity']=True
classVar='Decision'
toPrint=True
measurement=Entropy
find_best_split(df)

     Outlook Temperature  Humidity  Windy Decision
0      sunny         hot      0.90  False        n
1      sunny         hot      0.87   True        n
2   overcast         hot      0.93  False        p
3       rain        mild      0.89  False        p
4       rain        cool      0.80  False        p
5       rain        cool      0.59   True        n
6   overcast        cool      0.77   True        p
7      sunny        mild      0.91  False        n
8      sunny        cool      0.68  False        p
9       rain        mild      0.84  False        p
10     sunny        mild      0.72   True        p
11  overcast        mild      0.49   True        p
12  overcast         hot      0.74  False        p
13      rain        mild      0.86   True        n
Outlook 0.6935361388961919 Outlook 0.6935361388961919 nan
Temperature 0.9110633930116763 Outlook 0.6935361388961919 nan
Humidity 0.929967857760991 0.9
Humidity 0.9152077851647805 0.87
Humidity 0.9402859586706309 0.93
Humidity 0.8609819

('Outlook', 0.6935361388961919, nan)

<h3>Put it all together</h3>

In [36]:
def SplitNodes(df, Tree):
    attrib, b_score, b_value = find_best_split(df)

    Tree[attrib]=dict()

    if (np.isnan(b_value)):
        values=list(set(df[attrib]))
        if (len(values)==1):
            Tree[attrib][df[attrib][0]]=(list(df[classVar])[0], np.NAN)
            return
        
        for v in values:
            subdf=df[df[attrib]==v].drop(attrib, axis=1)
            if abs(measurement(subdf[classVar]))>1e-10: 
                Tree[attrib][v]={}
                SplitNodes(subdf, Tree[attrib][v])
            else:
                Tree[attrib][v]=list(subdf[classVar])[0]
    else:
        if df.shape[0]==1: 
            Tree[attrib][b_value]=(list(df[classVar])[0], b_value)
            return
        
        subdf=df[df[attrib]<=b_value].drop(attrib, axis=1)
        v='<'+str(b_value)  # which includes the <=
        if abs(measurement(subdf[classVar]))>1e-10: 
            Tree[attrib][v]={}
            SplitNodes(subdf, Tree[attrib][v])
        else:
            Tree[attrib][v]=list(subdf[classVar])[0]
            return

        subdf=df[df[attrib]> b_value].drop(attrib, axis=1)
        v='>'+str(b_value)
        if abs(measurement(subdf[classVar]))>1e-10: 
            Tree[attrib][v]={}
            SplitNodes(subdf, Tree[attrib][v])
        else:
            Tree[attrib][v]=list(subdf[classVar])[0]
            return
    del df

toPrint=False

df=pd.read_csv('playTennis_N.csv')
iscontinuous={col:False for col in df}
iscontinuous['Humidity']=True

classVar='Decision'
decisiontree=dict()
SplitNodes(df,decisiontree)
decisiontree

{'Outlook': {'rain': {'Windy': {False: 'p', True: 'n'}},
  'overcast': 'p',
  'sunny': {'Humidity': {'<0.72': 'p'}}}}

In [37]:
printTree(decisiontree)


  Outlook: 
    rain: 
      Windy: 
        False: p,  
        True: n,  
    overcast: p,  
    sunny: 
      Humidity: 
        <0.72: p,  

In [38]:
t0=time.time()
df=dataB = pd.read_csv('balance-scale.csv', sep= ',') 

X_train, X_test=sample(df, 0.3)

toPrint=False
iscontinuous={col:False for col in df}
classVar='Class'
decisiontree=dict()
SplitNodes(X_train,decisiontree)
print(time.time()-t0, 's')
decisiontree


1.0518178939819336 s


{'LeftDistance': {1: {'RightWeight': {1: {'LeftWeight': {1: {'RightDistance': {1: 'B',
        2: 'R',
        3: 'R',
        4: 'R',
        5: 'R'}},
      2: {'RightDistance': {1: 'L', 4: 'R', 5: 'R'}},
      3: {'RightDistance': {1: 'L', 2: 'L', 5: 'R'}},
      4: 'B',
      5: {'RightDistance': {3: 'L', 4: 'L', 5: 'B'}}}},
    2: {'RightDistance': {1: {'LeftWeight': {1: 'R', 3: 'L'}},
      2: {'LeftWeight': {1: 'R', 2: 'R', 4: 'B', 5: 'L'}},
      3: 'R',
      4: 'R',
      5: 'R'}},
    3: {'RightDistance': {1: {'LeftWeight': {1: 'R', 2: 'R', 5: 'L'}},
      2: 'R',
      3: 'R',
      4: 'R',
      5: 'R'}},
    4: {'RightDistance': {1: {'LeftWeight': {1: 'R', 2: 'R', 4: 'B', 5: 'L'}},
      2: 'R',
      3: 'R',
      4: 'R',
      5: 'R'}},
    5: 'R'}},
  2: {'RightWeight': {1: {'LeftWeight': {1: {'RightDistance': {1: 'L',
        3: 'R',
        4: 'R',
        5: 'R'}},
      2: {'RightDistance': {2: 'L', 3: 'L', 4: 'B', 5: 'R'}},
      3: 'L',
      4: 'L',
      5: 'L'

In [39]:
#the above code is close enough for a recursive prediction function, it take a dictionary and predict the testing point's 
def predict(dic, x_test):
    field=next(iter(dic))
    match=dic[field].get(x_test[field]) 
    if match is None: return
    if type(match) is dict:
        if toPrint: print(match)  # match is a shorted dictionary so it can be treated just like the dic
        return predict(match, x_test)
    else:
        if toPrint: print(match, 'is a leaf of type ', type(match))
        return match

X_test=df.iloc[0,:]

nbad=0
for i in range(X_train.shape[0]):
    predicted=predict(DecisionTree, X_train.iloc[i,:])
    if predicted!=X_train[classVar][i]:
        print("mismatch:",predicted!=X_train[classVar][i])
        nbad+=1
print(nbad, " is mismatched")

0  is mismatched


<h3>Prediction with continous variable</h3>

In [40]:
#the above code is close enough for a recursive prediction function, it take a dictionary and predict the testing point's 
def predict(dic, x_test):
    field=next(iter(dic))
    
    if iscontinuous[field]: 
        #print(x_test[field], dic[field], list(dic[field].values())[0])
        cond=next(iter(dic[field]))
        if x_test[field]<=float(cond[2:]): 
            match=dic[field].get(cond, defaultResult)
        else: 
            match=dic[field].get('>'+cond[1:], defaultResult) 
    else:
        match=dic[field].get(x_test[field], defaultResult) 
    if type(match) is dict:
        if toPrint: print(match)  # match is a shorted dictionary so it can be treated just like the dic
        return predict(match, x_test)
    else:
        if toPrint: print(match, 'is a leaf of type ', type(match))
        return match
    

dic1={'Outlook': {'sunny': {'Humidity': {'<0.72': 'p'}}, 'overcast': 'p', 'rain': {'Windy': {False: 'p', True: 'n'}}}}
dic1=decisiontree
defaultResult='n'
df=pd.read_csv('playTennis_N.csv', sep= ',') 
for i in range(len(df)): 
    print(predict(dic1, df.iloc[i,:]), df[classVar][i])

KeyError: 'LeftDistance'

In [41]:
df=pd.read_csv('playTennis_N.csv', sep= ',') 
toPrint=False
iscontinuous={col:False for col in df}
iscontinuous['Humidity']=True
classVar='Decision'
decisiontree=dict()
SplitNodes(df,decisiontree)
print(decisiontree)

predict(decisiontree, df.iloc[0,:])

{'Outlook': {'rain': {'Windy': {False: 'p', True: 'n'}}, 'overcast': 'p', 'sunny': {'Humidity': {'<0.72': 'p'}}}}


'n'

In [42]:
df=pd.read_csv('mushrooms.csv')
X_train, X_test=sample(df, 0.3)

t0=time.time()
toPrint=False
iscontinuous={col:False for col in df}
classVar='class'
decisiontree=dict()
SplitNodes(X_train,decisiontree)
print(time.time()-t0, 's')
print(decisiontree)

predicted=[predict(decisiontree, X_test.iloc[i,:]) for i in range(X_test.shape[0])]

KNNresultTable(predicted, list(X_test[classVar]))
    

0.5289251804351807 s
{'odor': {'l': 'e', 's': 'p', 'p': 'p', 'f': 'p', 'c': 'p', 'm': 'p', 'y': 'p', 'n': {'spore.print.color': {'b': 'e', 'w': {'habitat': {'l': {'cap.color': {'w': 'p', 'c': 'e', 'n': 'e', 'y': 'p'}}, 'g': 'e', 'w': 'e', 'd': {'gill.size': {'b': 'e', 'n': 'p'}}, 'p': 'e'}}, 'k': 'e', 'y': 'e', 'n': 'e', 'o': 'e', 'r': 'p', 'h': 'e'}}, 'a': 'e'}}

Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1 score: 1.0


pred,p,e
test,,
p,1128.0,0.0
e,0.0,1194.0
